# YOLOv4 Implementation

The goal of this notebook is the implementation of the YOLOv4 neuronal network for sign detection. For that, an installation and use of DarkNet Framework on Ubuntu (18.04) will be introduced. Then, the setup of the darknet repository will be modified depending on the nature of our specific goal and corresponding dataset. After that, the neural network will be trained on the German Traffic Sign Detection Benchmark (GTSDB) Dataset, then the best weights will be used to detect signs.

## Installation & Use DarkNet FRamework on UBuntu 18.04 (LTS)


Source: https://medium.com/@pavan.mksolution/installation-use-darknet-framework-on-ubuntu-18-04-lts-f90ac3041fed

Follow below steps for Installation :
1. Create your working directory as any name you can give.
2. Open Terminal and type command [sudo apt-get install git]
3. copy this command from the website : https://github.com/AlexeyAB/darknet.git

How to Use :

These are steps for CPU computation, just to check how our output will look like !
Once you clone the darknet to your directory (DarkNet Folder — root)
1. Open command prompt and run : `$ cd darknet`
2. Make the folder by running : `$ make`
3. Download Pre-Trained Model & simply copy it into your working directory (i.e darknet)
`!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137`
4. TEST : Run the command
`./darknet detect cfg/yolov3.cfg yolov3.weights data/dog.jpg`

## Modify the yolov4 repository 

Source: https://medium.com/ai-world/how-to-train-yolov4-for-custom-objects-detection-in-google-colab-1e934b8ef685

Now we have to make some changes to adapt the yolov4 file to the nature of the GTSDB dataset.

1. Open darknet-master folder which we have just downloaded and from that open cfg folder now in the cfg folder make a copy of the file yolo4-custom.cfg now rename the copy file to yolo-obj.cfg
    
    a. open the file yolo-obj.cfg and change max_batches to (classes*2000),if you have 6 object classes change max_batches=12000.

    b. Then change the line steps to (0.8*max_batches ,0.9*max_batches) ie; if you have 6 classes steps=9600,10800.

    c. set network size width=416 height=416

    d. change line classes=80 to your number of objects in each of 3 yolo layers.

    e. change [filters=255] to filters=(classes + 5)x3 in the 3 convolutional layer immediately before each 3 yolo layers.If you have 6 classes filters=33
2. Download the pre trained weights from the link [yolov4.conv.137](https://drive.google.com/file/d/1JKF-bdIklxOOVy-2Cr5qdvjgGpmGfcbp/view) and save it in the darknet-master folder

3. Open wordpad and type the name of each object in separate lines and save the file as obj.names in darknet-master->data folder

0 = speed limit 20 (prohibitory)

1 = speed limit 30 (prohibitory)

2 = speed limit 50 (prohibitory)

3 = speed limit 60 (prohibitory)

4 = speed limit 70 (prohibitory)

5 = speed limit 80 (prohibitory)

6 = restriction ends 80 (other)

7 = speed limit 100 (prohibitory)

8 = speed limit 120 (prohibitory)

9 = no overtaking (prohibitory)

10 = no overtaking (trucks) (prohibitory)

11 = priority at next intersection (danger)

12 = priority road (other)

13 = give way (other)

14 = stop (other)

15 = no traffic both ways (prohibitory)

16 = no trucks (prohibitory)

17 = no entry (other)

18 = danger (danger)

19 = bend left (danger)

20 = bend right (danger)

21 = bend (danger)

22 = uneven road (danger)

23 = slippery road (danger)

24 = road narrows (danger)

25 = construction (danger)

26 = traffic signal (danger)

27 = pedestrian crossing (danger)

28 = school crossing (danger)

29 = cycles crossing (danger)

30 = snow (danger)

31 = animals (danger)

32 = restriction ends (other)

33 = go right (mandatory)

34 = go left (mandatory)

35 = go straight (mandatory)

36 = go right or straight (mandatory)

37 = go left or straight (mandatory)

38 = keep right (mandatory)

39 = keep left (mandatory)

40 = roundabout (mandatory)

41 = restriction ends (overtaking) (other)

42 = restriction ends (overtaking (trucks)) 
(other)

4. Create file obj.data in the folder darknet-master->data, containing the given text (replace classes = number of objects)


```
classes = 43
train = data/train.txt
valid = data/test.txt
names = data/obj.names
backup = backup/
```



5. Create a folder in the directory darknet-master->data named obj now inside this obj folder put all your images and the respective txt files you got from labeling the dataset (**ONLY NECESSARY FOR THE TRAINING, NOT FOT THE DETECTION**)

6. Now we have to create a train.txt file.This file directs to all training images as shown in the below picture.The easiest way to achieve this is store all images in a folder in computer open command prompt navigate to the folder using ‘cd’ and type command ‘ls’ if in linux and ‘dir’ if in windows.This will display all image names copy that and paste in text file and add ‘data/obj/train’ to each line for this Find and replace option could be used.The train.txt file is stored in the darknet-master->data folder. (**ONLY NECESSARY FOR THE TRAINING, NOT FOT THE DETECTION**)

```
data/obj/Train/00179.ppm
data/obj/Train/00382.ppm
data/obj/Train/00353.ppm
data/obj/Train/00265.ppm
..
..
```
**The same thing must be done for the validation set.**

7. In the darknet-master folder open Makefile in wordpad and change GPU=1,CUDNN=1,OPENCV=1 as shown in the following picture.This is done to make the training on GPU.

```
GPU=1
CUDNN=1
CUDNN_HALF=0
OPENCV=1
AVX=0
OPENMP=0
LIBSO=0
ZED_CAMERA=0 # ZED SDK 3.0 and above
ZED_CAMERA_v2_8=0 # ZED SDK 2.X
```



## Training on the GTSDB Dataset and Detecting Signs

Now the modifications have been made, the training can start.

In [ ]:
./darknet detector train data/obj.data cfg/yolo-obj.cfg yolov4.conv.137 -dont_show -map

Now that the network is trained it is ready to be used for detection.

let's take 'data/obj/Test/00601.ppm' as an example here:

In [ ]:
# This code has to be executed one time only
#coco.names is hardcoded somewhere in the detector
%cp data/obj.names data/coco.names

In [ ]:
./darknet detect cfg/yolo-obj.cfg backup/custom-yolov4-detector_best.weights data/obj/Test/00601.ppm -dont-show

To be able to see the results of the detection use the following code:

In [ ]:
#define utility function
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  #plt.rcParams['figure.figsize'] = [10, 5]
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

In [ ]:
./darknet detect cfg/yolo-obj.cfg backup/custom-yolov4-detector_best.weights data/obj/Test/00601.ppm -dont-show
imShow('predictions.jpg')

## Implementing in the hardware

To implement the yolov4 model in the hardware, some of the steps above are not necessary. The dataset (image and annotations) shouldnt be uploaded into the machine, since we are using the best weights for detection. The text files containing the images of the training and validation dataset aren't necessary for the detection either.

Additional sources:
https://github.com/AlexeyAB/darknet
https://colab.research.google.com/drive/1mzL6WyY9BRx4xX476eQdhKDnd_eixBlG#scrollTo=x-_E3O5Mf4Mf
https://benchmark.ini.rub.de/
https://medium.com/ai-world/how-to-train-yolov4-for-custom-objects-detection-in-google-colab-1e934b8ef685